In [ ]:
import torch
import torch.nn as nn
import math

class MultiHeadAttention(nn.Module):
    def __init__(self, d_model: int, num_heads: int, dropout: float = 0.1):
        """
        Args:
            d_model (int): モデルの隠れ層の次元数
            num_heads (int): ヘッドの数
            dropout (float, optional): ドロップアウト率. Defaults to 0.1.
        """

        super().__init__()

        # d_modelがnum_headsで割り切れることを確認
        assert d_model % num_heads == 0, "d_model must be divisible by num_heads"
        
        self.d_model = d_model
        self.num_heads = num_heads
        self.d_k = d_model // num_heads # 各ヘッドの次元数

        # Q, K, Vの線形変換
        # 実際には全ヘッド分を一度に計算するため、出力次元はd_modelのまま
        self.w_q = nn.Linear(d_model, d_model)
        self.w_k = nn.Linear(d_model, d_model)
        self.w_v = nn.Linear(d_model, d_model)

        self.fc_out = nn.Linear(d_model, d_model)
        self.dropout = nn.Dropout(dropout)
    
    def forward(self, query: torch.Tensor, key: torch.Tensor, value: torch.Tensor, mask: torch.Tensor = None) -> torch.Tensor:
        """

        Args:
            query (torch.Tensor): [batch_size, seq_len, d_model]
            key (torch.Tensor):   [batch_size, seq_len, d_model]
            value (torch.Tensor): [batch_size, seq_len, d_model]
            mask (torch.Tensor, optional): [batch_size, 1, 1, seq_len] または [batch_size, 1, seq_len, seq_len]
                                           (0: マスクなし、1: マスクありなどの定義によるが、ここでは加算マスクを想定)
                                           Defaults to None.

        Returns:
            torch.Tensor: [batch_size, seq_len, d_model]
        """
        batch_size = query.size(0)

        # 1. 線形変換
        # [batch_size, seq_len, d_model] -> [batch_size, seq_len, num_heads]
        Q = self.w_q(query)
        K = self.w_k(key)
        V = self.w_v(value)

        # 2. ヘッドの分割
        # [batch_size, seq_len, num_heads] -> [batch_size, seq_len, num_heads, d_k]
        # その後、計算しやすいようにヘッドの次元を先頭に移動させる(転置) -> [batch_size, num_heads, seq_len, d_k]
        Q = Q.view(batch_size, -1, self.num_heads, self.d_k).transpose(1, 2)
        K = K.view(batch_size, -1, self.num_heads, self.d_k).transpose(1, 2)
        V = V.view(batch_size, -1, self.num_heads, self.d_k).transpose(1, 2)

        # 3. Scaled Dot-Product Attention
        # 3.1. スコアの計算 Q * K^T / sqrt(d_k)
        # Q: [..., seq_len_q, d_k], K^T: [..., d_k, seq_len_k] -> scores: [..., seq_len_q, seq_len_k]
        scores = torch.matmul(Q, K.transpose(-2, -1)) / math.sqrt(self.d_k)

        # 3.2. マスクの適用(optional)
        if mask is not None:
            # ここではマスクが0の場所を非常に小さい値(-1e9)でマスクすると仮定
            # 実装により1と0の定義が異なる場合があるため注意する
            # 非常に小さい値(-1e9)で埋めることで、softmax後にほぼ0になるようにする
            # scores = scores.masked_fill(mask == 0, -1e9)
            scores = scores + mask # 加算マスクの場合 (maskが0の場所に-1e9が入っている想定)
        
        # 3.3. softmax & dropout
        attention_weights = torch.softmax(scores, dim=-1)
        attention_weights = self.dropout(attention_weights)

        # 3.4. Valueとの積
        # attention_weights: [..., seq_len_q, seq_len_k] * V: [..., seq_len_k, d_k] -> [...,seq_lenn_q, d_k]
        output = torch.matmul(attention_weights, V)

        # 4. ヘッドの結合
        # [batch_size, num_beads, seq_len, d_k] -> [batch_size, seq_len, num_heads, d_k]
        output = output.transpose(1, 2).contiguous()

        # [batch_size, seq_len, d_model]に戻す
        output = output.view(batch_size, -1, self.d_model)

        # 5. 線形変換
        output = self.fc_out(output)
        return output

In [ ]:
def verify_multi_head_attention():
    batch_size = 2
    seq_len = 10
    d_model = 512
    num_heads = 8

    mha = MultiHeadAttention(d_model, num_heads)

    # ダミー入力の作成
    # self-attentionなのでkey, valueもqueryと同じにする
    x = torch.rand(batch_size, seq_len, d_model)
    output = mha(x, x, x)

    print("Input shape:", x.shape)
    print("Output shape:", output.shape)

    # 出力の形状が入力と同じであることを確認
    assert output.shape == x.shape, "Output shape must match input shape"
    print("MultiHeadAttention verification passed!")

verify_multi_head_attention()